In [1]:
import os
import pandas as pd
from collections import defaultdict

In [2]:
username = "DRISHTI"

In [3]:
root_dir = fr"C:\Users\{username}\Desktop\Chest X ray data"
csv_path = os.path.join(root_dir, "Data_Entry_2017.csv")

In [5]:
df = pd.read_csv(csv_path)
df

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,0.168,NaN
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,0.168,NaN
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168,NaN
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168,NaN


In [6]:
# Create a mapping from image name to disease label(s)
image_to_disease = dict(zip(df['Image Index'], df['Finding Labels']))

# Prepare a dict to collect disease labels per folder
folder_disease_map = defaultdict(list)

# === PROCESS FOLDERS ===
for i in range(1, 13):
    folder_name = f'images_{i:03d}'
    image_folder = os.path.join(root_dir, folder_name, 'images')
    
    if not os.path.isdir(image_folder):
        print(f"[Warning] Folder not found: {image_folder}")
        continue
    
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]
    
    for image in image_files:
        if image in image_to_disease:
            folder_disease_map[folder_name].append(image_to_disease[image])
        else:
            print(f"[Missing in CSV] {image} not found in CSV")

# === SUMMARY REPORT ===
print("\n=== Folder-wise Disease Summary ===")
for folder, diseases in folder_disease_map.items():
    unique_diseases = set(diseases)
    print(f"\n{folder} contains {len(diseases)} images with {len(unique_diseases)} unique label(s):")
    for disease in unique_diseases:
        print(f"  - {disease}")


=== Folder-wise Disease Summary ===

images_001 contains 4999 images with 209 unique label(s):
  - Atelectasis|Cardiomegaly|Consolidation|Effusion
  - Effusion|Fibrosis
  - Effusion|Fibrosis|Mass|Nodule|Pleural_Thickening
  - Consolidation|Fibrosis
  - Edema|Effusion
  - Fibrosis
  - Effusion|Pneumonia|Pneumothorax
  - Consolidation|Infiltration|Pneumothorax
  - Edema|Effusion|Infiltration
  - Emphysema|Infiltration|Pleural_Thickening|Pneumothorax
  - Infiltration|Nodule|Pneumothorax
  - Cardiomegaly|Effusion|Hernia
  - Mass|Pleural_Thickening
  - Consolidation
  - Edema|Infiltration|Nodule
  - Cardiomegaly|Effusion|Infiltration
  - Atelectasis|Emphysema
  - Cardiomegaly|Effusion|Pneumonia
  - Consolidation|Nodule|Pleural_Thickening
  - Effusion|Infiltration|Pleural_Thickening|Pneumonia
  - Infiltration|Nodule|Pleural_Thickening
  - Consolidation|Mass
  - Atelectasis|Infiltration
  - Cardiomegaly|Edema
  - Edema|Effusion|Pneumonia
  - Atelectasis|Consolidation|Effusion|Mass|Pleural_Th

In [10]:
import os
import pandas as pd
import shutil

# === CONFIGURATION ===
username = "DRISHTI"  # <-- change this
root_dir = fr"C:\Users\{username}\Desktop\Chest X ray data"
csv_path = os.path.join(root_dir, "Data_Entry_2017.csv")

# Destination folders
pneumonia_dir = os.path.join(root_dir, "pneumonia")
normal_dir = os.path.join(root_dir, "normal")

# Create folders if they don't exist
os.makedirs(pneumonia_dir, exist_ok=True)
os.makedirs(normal_dir, exist_ok=True)

# === LOAD CSV AND MAP IMAGE TO LABEL ===
df = pd.read_csv(csv_path)
image_to_disease = dict(zip(df['Image Index'], df['Finding Labels']))

# === PROCESS IMAGES ===
for i in range(1, 13):
    folder_name = f'images_{i:03d}'
    image_folder = os.path.join(root_dir, folder_name, 'images')
    
    if not os.path.isdir(image_folder):
        print(f"[Skip] Folder not found: {image_folder}")
        continue
    
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]
    
    for image in image_files:
        label = image_to_disease.get(image, None)
        if label is None:
            continue

        src_path = os.path.join(image_folder, image)

        # Only "Pneumonia" (exact match)
        if label == "Pneumonia":
            dst_path = os.path.join(pneumonia_dir, image)
            shutil.copy2(src_path, dst_path)

        # Only "No Finding"
        elif label == "No Finding":
            dst_path = os.path.join(normal_dir, image)
            shutil.copy2(src_path, dst_path)

print("✅ Images successfully copied to 'pneumonia' and 'normal' folders.")


✅ Images successfully copied to 'pneumonia' and 'normal' folders.
